# Pre-processing SCAN-seq2 data

Here we pre-process SCAN-seq2 data published [here](https://www.nature.com/articles/s41421-022-00500-4)

- HepG2 SCAN-seq2 (single-cell nanopore-based): [GSE203561](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE203561).
File with UMI counts table: GSM6176321_gene_counts_9CL.txt
Metadata from the paper: 41421_2022_500_MOESM2_ESM.xlsx

- K562 SCAN-seq2: We used K562 cells contained in GSM6176321_gene_counts_9CL.txt and GSM6176325_gene_counts_UMI_200.txt

In [ ]:
%matplotlib inline

In [ ]:
import scanpy as sc
import anndata as ad
import scprep as scp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [ ]:
HepG2_folder='./HepG2/'
K562_folder='./K562/'
input_folder='../../ANALYSIS_FEB_2023_RIBO/'

In [ ]:
sc.__version__

In [ ]:
# Read the metadata
metadata=pd.read_excel("./Metadata/41421_2022_500_MOESM2_ESM.xlsx")

In [ ]:
metadata.Library.value_counts()

In [ ]:
metadata=metadata.set_index('Cell_ID')

In [ ]:
# Read the data
adata_9CL=ad.read_csv('./Data/GSM6176321_gene_counts_9CL.txt',delimiter='\t')
adata_9CL=adata_9CL.T
adata_4CL=ad.read_csv('./Data/GSM6176323_gene_counts_4CL.txt',delimiter='\t')
adata_4CL=adata_4CL.T
adata_9CL_Mix=ad.read_csv('./Data/GSM6176322_gene_counts_9CL_Mix.txt',delimiter='\t')
adata_9CL_Mix=adata_9CL_Mix.T
adata_UMI_200=ad.read_csv('./Data/GSM6176325_gene_counts_UMI_200.txt',delimiter='\t')
adata_UMI_200=adata_UMI_200.T
adata_UMI_100=ad.read_csv('./Data/GSM6176324_gene_counts_UMI_100.txt',delimiter='\t')
adata_UMI_100=adata_UMI_100.T

In [ ]:
adata=ad.concat([adata_9CL,adata_4CL,adata_9CL_Mix,adata_UMI_200,adata_UMI_100])

In [ ]:
(adata.obs_names==metadata.index).all()

In [ ]:
print(len(adata.obs_names),len(metadata),len(set(adata.obs_names).intersection(set(metadata.index))))

In [ ]:
metadata=metadata.reindex(adata.obs_names)
adata.obs=metadata

In [ ]:
adata.var['ercc'] = adata.var_names.str.startswith('ERCC')  # annotate the group of mitochondrial genes as 'mt'

In [ ]:
adata=adata[:,adata.var_names[:35582]].copy()
print(adata)
adata = adata[:,~adata.var['ercc']].copy()
print(adata)

In [ ]:
adata_K562=adata[adata.obs.Cell_Line=='K562'].copy()
print(adata_K562.n_obs)
adata_K562=adata_K562[adata_K562.obs.Pass_QC==1].copy()
print(adata_K562.n_obs)
adata_HepG2=adata[adata.obs.Cell_Line=='HepG2'].copy()
print(adata_HepG2.n_obs)
adata_HepG2=adata_HepG2[adata_HepG2.obs.Pass_QC==1].copy()
print(adata_HepG2.n_obs)

In [ ]:
# Save the names of
# - HepG2 9CL cells
# - K562 9CL cells
# - K562 UMI200 cells
np.savetxt('HepG2_9CL_cellnames.txt',
           np.c_[list(metadata[(metadata.Cell_Line=='HepG2') & (metadata.Library=='9CL')].index)],fmt="%s")
np.savetxt('K562_9CL_cellnames.txt',
           np.c_[list(metadata[(metadata.Cell_Line=='K562') & (metadata.Library=='9CL')].index)],fmt="%s")
np.savetxt('K562_UMI200_cellnames.txt',
           np.c_[list(metadata[(metadata.Cell_Line=='K562') & (metadata.Library=='UMI_200')].index)],fmt="%s")

In [ ]:
np.savetxt('HepG2_genenames.txt',
           np.c_[list(adata_HepG2.var_names)],fmt="%s")

In [ ]:
sc.pp.filter_genes(adata_K562,min_cells=10)
adata_K562
sc.pp.filter_genes(adata_HepG2,min_cells=10)
adata_HepG2

In [ ]:
adata_K562.obs.Organism.value_counts()

In [ ]:
sc.pl.violin(adata_K562,keys=['Mapped_percent','Mapped_Reads','UMI_count','Gene_Detected'],groupby='Library')

In [ ]:
adata_HepG2.obs.Library.value_counts()

In [ ]:
sc.pl.violin(adata_HepG2,keys=['Mapped_percent','Mapped_Reads','UMI_count','Gene_Detected'],groupby='Library')

# Cluster K562 cells

In [ ]:
sc.pp.normalize_total(adata_K562)
sc.pp.log1p(adata_K562)
sc.pp.highly_variable_genes(adata_K562,max_mean=10,batch_key='Library')
adata_K562_high_var = adata_K562[:, adata_K562.var.highly_variable]
sc.pp.scale(adata_K562_high_var, max_value=10)
sc.tl.pca(adata_K562_high_var, svd_solver='arpack')
sc.pp.neighbors(adata_K562_high_var)
#sc.pl.paga(adata_CM, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata_K562_high_var)
sc.pl.umap(adata_K562_high_var,color='Library')

In [ ]:
adata_K562.obs.Library.value_counts()

K562: We run the analysis separately for 9CL and UMI_200

In [ ]:
adata_K562_9CL=adata[adata.obs.Cell_Line=='K562'].copy()
adata_K562_9CL=adata_K562_9CL[adata_K562_9CL.obs.Library=='9CL'].copy()
adata_K562_9CL=adata_K562_9CL[adata_K562_9CL.obs.Pass_QC==1].copy()

adata_K562_9CL

In [ ]:
adata_K562_UMI_200=adata[adata.obs.Cell_Line=='K562'].copy()
adata_K562_UMI_200=adata_K562_UMI_200[adata_K562_UMI_200.obs.Library=='UMI_200'].copy()
adata_K562_UMI_200=adata_K562_UMI_200[adata_K562_UMI_200.obs.Pass_QC==1].copy()

adata_K562_UMI_200

# Cluster HepG2 cells

In [ ]:
sc.pp.normalize_total(adata_HepG2)
sc.pp.log1p(adata_HepG2)
sc.pp.highly_variable_genes(adata_HepG2,max_mean=10,batch_key='Library')
adata_HepG2_high_var = adata_HepG2[:, adata_HepG2.var.highly_variable]
sc.pp.scale(adata_HepG2_high_var, max_value=10)
sc.tl.pca(adata_HepG2_high_var, svd_solver='arpack')
sc.pp.neighbors(adata_HepG2_high_var)
#sc.pl.paga(adata_CM, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata_HepG2_high_var)
sc.pl.umap(adata_HepG2_high_var,color='Library')

In [ ]:
adata_HepG2.obs.Library.value_counts()

We consider the 9CL library for HepG2 cells

In [ ]:
adata_HepG2_9CL=adata[adata.obs.Cell_Line=='HepG2'].copy()
adata_HepG2_9CL=adata_HepG2_9CL[adata_HepG2_9CL.obs.Library=='9CL'].copy()
adata_HepG2_9CL=adata_HepG2_9CL[adata_HepG2_9CL.obs.Pass_QC==1].copy()

adata_HepG2_9CL

# Load and pre-process data

We convert the gene names in all datasets to match the ENSEMBL 107 version with which we compute catRAPID (and processed eCLIP data)

## Utils for gene name conversion

In [ ]:
# Load the fasta file with the canonical isoforms
from Bio import SeqIO
gname=[]
gid=[]
f_open = open("/Users/jonathan/Desktop/IIT/INTERACTomics/ENCODE_eCLIP_DATA/transcriptomes/hsapiens_gene_ensembl_107_canonical.fa", "rU")
for rec in SeqIO.parse(f_open, "fasta"):
    myid = rec.id
    gname.append(myid.split('|')[4])
    gid.append(myid.split('|')[0])

In [ ]:
adata_list=[adata_K562_9CL,adata_K562_9CL_mix,adata_K562_UMI_200,adata_HepG2_9CL]
labels=['K562_9CL_SCANseq2','K562_UMI200_SCANseq2','HepG2_9CL_SCANseq2']

In [ ]:
# Retrieve the ensembl gene ID
import os
out_dir=os.getcwd()+'/missing_genes/'
if os.path.isdir(out_dir)==False:
    os.mkdir(out_dir)

missing_genes_list=[]
for (lab,adata) in zip(labels,adata_list):
    print(lab,len(gname),len(adata.var_names),
      len(set(gname).intersection(set(adata.var_names))))
    missing=list(set(adata.var_names)-set(gname).intersection(set(adata.var_names)))
    missing_genes_list.append(missing)
    np.savetxt(out_dir+'missing'+lab+'.txt',np.c_[missing],fmt='%s')

In [ ]:
# Load the mapping gene name/ENSEMBL GENE ID for each cell type
mapping=pd.read_csv(out_dir+'mapping.csv',index_col=0)

In [ ]:
def map_gene_names(adata,mapping):
    print('before')
    print(len(gname),len(adata.var_names),
      len(set(gname).intersection(set(adata.var_names))))
    j=1
    no_name_genes=[]
    mylist=list(adata.var_names)
    
    for i in range(len(mylist)):
        if mylist[i]=='AARS':
            mylist[i]='AARS1'
        if mylist[i]=='TROVE2':
            mylist[i]='RO60'
    for i in range(len(mapping.index)):
        # Retrieved ENSEMBL gene ID
        mygene_id=mapping.loc[mapping.index[i],'V1']
    
        # Find the corresponding gene name in the fasta from gencodeV41
        if mygene_id in gid:
            new_gene_name=gname[gid.index(mygene_id)]
            if new_gene_name=='':
                new_gene_name=mygene_id
                no_name_genes.append(mygene_id)
        
            # Find the index of the old gene name in var_names
            idx=mylist.index(mapping.index[i])
            if new_gene_name not in mylist:
                mylist[idx]=new_gene_name
        else:
            j+=1
    adata.var_names=mylist
    print('after')
    print(len(gname),len(adata.var_names),
      len(set(gname).intersection(set(adata.var_names))))
    print('-'*50)
    return adata;

In [ ]:
mapping.loc['ABCF2.H2BE1']

In [ ]:
mapping.rename(index={'ABCF2.H2BE1':'ABCF2-H2BE1'},inplace=True)
mapping.rename(index={'H3.2':'H3-2'},inplace=True)
mapping.rename(index={'LINC00266.2P':'LINC00266-2P'},inplace=True)

In [ ]:
adata_HepG2_9CL=map_gene_names(adata_HepG2_9CL,mapping)
adata_K562_9CL=map_gene_names(adata_K562_9CL,mapping)
adata_K562_9CL_mix=map_gene_names(adata_K562_9CL_mix,mapping)
adata_K562_UMI_200=map_gene_names(adata_K562_UMI_200,mapping)

In [ ]:
sc.pp.filter_genes(adata_HepG2_9CL, min_cells=int(0.1*adata_HepG2_9CL.n_obs))
print(adata_HepG2_9CL)

adata_HepG2_9CL.raw=adata_HepG2_9CL
sc.pp.normalize_total(adata_HepG2_9CL,inplace=True)
adata_HepG2_9CL_for_ARACNe=adata_HepG2_9CL.copy()

sc.pp.log1p(adata_HepG2_9CL)

In [ ]:
sc.pp.filter_genes(adata_K562_9CL, min_cells=int(0.1*adata_K562_9CL.n_obs))
print(adata_K562_9CL)

adata_K562_9CL.raw=adata_K562_9CL
sc.pp.normalize_total(adata_K562_9CL,inplace=True)
adata_K562_9CL_for_ARACNe=adata_K562_9CL.copy()

sc.pp.log1p(adata_K562_9CL)

In [ ]:
sc.pp.filter_genes(adata_K562_UMI_200, min_cells=int(0.1*adata_K562_UMI_200.n_obs))
print(adata_K562_UMI_200)

adata_K562_UMI_200.raw=adata_K562_UMI_200
sc.pp.normalize_total(adata_K562_UMI_200,inplace=True)
adata_K562_UMI_200_for_ARACNe=adata_K562_UMI_200.copy()

sc.pp.log1p(adata_K562_UMI_200)

## Remove mitochondrial genes

In [ ]:
def FilterMito(adata):
    mito_genes = adata.var_names.str.startswith('MT-')
    genes_to_keep = np.invert(mito_genes)
    print('before',adata)
    adata = adata[:,genes_to_keep].copy()
    print('after',adata)
    return adata;

In [ ]:
adata_HepG2_9CL=FilterMito(adata_HepG2_9CL)
adata_K562_9CL=FilterMito(adata_K562_9CL)
adata_K562_UMI_200=FilterMito(adata_K562_UMI_200)

# Diffusion pseudotime

## HepG2

In [ ]:
sc.pp.highly_variable_genes(adata_HepG2_9CL,max_mean=10,n_top_genes=2000)  #calculate highly variable genes
adata_HepG2_9CL_high = adata_HepG2_9CL[:,adata_HepG2_9CL.var['highly_variable']==True].copy()  #select only highly variable genes
sc.pp.scale(adata_HepG2_9CL_high,max_value=10)
sc.tl.pca(adata_HepG2_9CL_high,svd_solver='arpack')
sc.pl.pca_overview(adata_HepG2_9CL_high)

In [ ]:
sc.pp.neighbors(adata_HepG2_9CL_high, n_neighbors=10, n_pcs=10)
sc.tl.umap(adata_HepG2_9CL_high)
sc.tl.leiden(adata_HepG2_9CL_high)
sc.pl.umap(adata_HepG2_9CL_high,color='leiden')

In [ ]:
adata_HepG2_9CL_high.uns['iroot'] = np.argmin(adata_HepG2_9CL_high.obsm['X_umap'][:,0])

# Create the diffusion map
sc.tl.diffmap(adata_HepG2_9CL_high)

# Run Diffusion Pseudotime with 0 branching event
sc.tl.dpt(adata_HepG2_9CL_high)

# Grab the output and store in our metadata DataFrame
adata_HepG2_9CL_high.obs['dpt'] = adata_HepG2_9CL_high.obs['dpt_pseudotime']
adata_HepG2_9CL_high.obs.head()

## K562

### 9CL

In [ ]:
sc.pp.highly_variable_genes(adata_K562_9CL,max_mean=10,n_top_genes=2000)  #calculate highly variable genes
adata_K562_9CL_high = adata_K562_9CL[:,adata_K562_9CL.var['highly_variable']==True].copy()  #select only highly variable genes
sc.pp.scale(adata_K562_9CL_high,max_value=10)
sc.tl.pca(adata_K562_9CL_high,svd_solver='arpack')
sc.pl.pca_overview(adata_K562_9CL_high)

In [ ]:
sc.pp.neighbors(adata_K562_9CL_high, n_neighbors=15, n_pcs=10)
sc.tl.umap(adata_K562_9CL_high)

In [ ]:
sc.tl.leiden(adata_K562_9CL_high)
sc.pl.umap(adata_K562_9CL_high,color='leiden')

In [ ]:
adata_K562_9CL_high.uns['iroot'] = np.argmax(adata_K562_9CL_high.obsm['X_umap'][:,0])

# Create the diffusion map
sc.tl.diffmap(adata_K562_9CL_high)

# Run Diffusion Pseudotime with 1 branching event
sc.tl.dpt(adata_K562_9CL_high)

# Grab the output and store in our metadata DataFrame
adata_K562_9CL_high.obs['dpt'] = adata_K562_9CL_high.obs['dpt_pseudotime']
adata_K562_9CL_high.obs.head()

### UMI_200

In [ ]:
sc.pp.highly_variable_genes(adata_K562_UMI_200,max_mean=10,n_top_genes=2000)  #calculate highly variable genes
adata_K562_UMI_200_high = adata_K562_UMI_200[:,adata_K562_UMI_200.var['highly_variable']==True].copy()  #select only highly variable genes
sc.pp.scale(adata_K562_UMI_200_high,max_value=10)
sc.tl.pca(adata_K562_UMI_200_high,svd_solver='arpack')
sc.pl.pca_overview(adata_K562_UMI_200_high)

In [ ]:
sc.pp.neighbors(adata_K562_UMI_200_high, n_neighbors=10, n_pcs=10)
sc.tl.umap(adata_K562_UMI_200_high)
sc.tl.leiden(adata_K562_UMI_200_high)
sc.pl.umap(adata_K562_UMI_200_high,color='leiden')

In [ ]:
adata_K562_UMI_200_high.uns['iroot'] = np.argmin(adata_K562_UMI_200_high.obsm['X_umap'][:,0])

# Create the diffusion map
sc.tl.diffmap(adata_K562_UMI_200_high)

# Run Diffusion Pseudotime with 0 branching event
sc.tl.dpt(adata_K562_UMI_200_high)

# Grab the output and store in our metadata DataFrame
adata_K562_UMI_200_high.obs['dpt'] = adata_K562_UMI_200_high.obs['dpt_pseudotime']
adata_K562_UMI_200_high.obs.head()

### Check the overlap between the highly variable genes

In [ ]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union

In [ ]:
import seaborn as sns

protocols=['9CL','9CL_Mix','UMI_200']
ct='K562'
adatas=[adata_K562_9CL_high,adata_K562_9CL_mix_high,adata_K562_UMI_200_high]

jaccard=np.zeros((len(adatas),len(adatas)))

i=0
for (adata1,prot1) in zip(adatas,protocols):
    j=0
    for (adata2,prot2) in zip(adatas,protocols):
        jaccard[i,j]=jaccard_similarity(list(adata1.var_names),list(adata2.var_names))
        j+=1
    i+=1

In [ ]:
# Getting the Upper Triangle of the co-relation matrix
matrix = np.triu(jaccard)

fig,ax =plt.subplots()
ax.set_title('K562')
# using the upper triangle matrix as mask 
sns.heatmap(jaccard, annot=True, mask=matrix,ax=ax,
           xticklabels=protocols,yticklabels=protocols)
plt.show(),plt.close()

## Save pseudotime data

In [ ]:
pseudo_folder=input_folder+'PseudoTime/'

In [ ]:
pseudo_df=pd.DataFrame(data=adata_HepG2_9CL_high.obs['dpt'], index=adata_HepG2_9CL_high.obs_names)
pseudo_df.to_csv(pseudo_folder+'HepG2_9CL_SCANseq2_PseudoTime.csv')

In [ ]:
pseudo_df=pd.DataFrame(data=adata_K562_9CL_high.obs['dpt'], index=adata_K562_9CL_high.obs_names)
pseudo_df.to_csv(pseudo_folder+'K562_9CL_SCANseq2_PseudoTime.csv')

In [ ]:
pseudo_df=pd.DataFrame(data=adata_K562_9CL_mix_high.obs['dpt'], index=adata_K562_9CL_mix_high.obs_names)
pseudo_df.to_csv(pseudo_folder+'K562_9CLMix_SCANseq2_PseudoTime.csv')

pseudo_df=pd.DataFrame(data=adata_K562_UMI_200_high.obs['dpt'], index=adata_K562_UMI_200_high.obs_names)
pseudo_df.to_csv(pseudo_folder+'K562_UMI200_SCANseq2_PseudoTime.csv')

## Save the processed data for gene selection

In [ ]:
proc_folder=input_folder+'processed_data/'

In [ ]:
adata_HepG2_9CL.write_h5ad(proc_folder+'processed_HepG2_9CL_SCANseq2.h5ad')
adata_HepG2_9CL_for_ARACNe.write_h5ad(proc_folder+'processed_HepG2_9CL_SCANseq2_ARACNe.h5ad')

adata_K562_UMI_200.write_h5ad(proc_folder+'processed_K562_UMI200_SCANseq2.h5ad')
adata_K562_UMI_200_for_ARACNe.write_h5ad(proc_folder+'processed_K562_UMI200_SCANseq2_ARACNe.h5ad')

In [ ]:
adata_K562_9CL.write_h5ad(proc_folder+'processed_K562_9CL_SCANseq2.h5ad')
adata_K562_9CL_for_ARACNe.write_h5ad(proc_folder+'processed_K562_9CL_SCANseq2_ARACNe.h5ad')